In [135]:
import pandas as pd
import numpy as np

In [136]:
df = pd.read_csv('.\AccidentesAviones.csv')

Eliminamos la primer columna que tiene un índice innecesario

In [137]:
df = df.drop(df.columns[0], axis=1)

Cambiamos los nombres de las columnas

In [138]:
nuevos_nombres = ['Date','Time','Location','Operator','Flight','Route','Type','Registration','Serial','Aboard','Passenger_Aboard','Crew_Aboard','Fatalities','Passenger_Fatalities','Crew_Fatalities','Ground_Fatalities','Summary']
df.columns = nuevos_nombres

Reemplazamos los valores '?' por valores vacíos (NaN)

In [139]:
df.replace('?', np.nan, inplace=True)

Convertimos la columna Date en formato fecha

In [140]:
df['Date'] = pd.to_datetime(df['Date'])

Convertimos la columna Time en formato hora

In [141]:
df['Time'] = pd.to_datetime(df['Time'], format='%H%M', errors='coerce')
df['Time'] = df['Time'].dt.strftime('%H:00')

Convertimos algunas columnas a formato INT

In [142]:
df['Aboard'] = df['Aboard'].fillna(0).astype(int)
df['Fatalities'] = df['Fatalities'].fillna(0).astype(int)
df['Crew_Fatalities'] = df['Crew_Fatalities'].fillna(0).astype(int)

In [143]:
df.head(2)

,Date,Time,Location,Operator,Flight,Route,Type,Registration,Serial,Aboard,Passenger_Aboard,Crew_Aboard,Fatalities,Passenger_Fatalities,Crew_Fatalities,Ground_Fatalities,Summary
0,1908-09-17,17:00,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army flyer flown by Orville Wright nos..."
1,1909-09-07,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1,0,1,1,0,0,0,"Eugene Lefebvre was the first pilot to ever be killed in an air accident, af..."


Separamos la columna Location en 2 columnas que se dividan en City/State y Country

In [144]:
df[['City/State', 'Country']] = df['Location'].str.rsplit(', ', n=1, expand=True)

In [145]:
# Lista de estados de Estados Unidos
estados_usa = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut",
    "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan",
    "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
    "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma",
    "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee",
    "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

def corregir_ciudad_estado(row):
    if row['Country'] in estados_usa:
        row['City/State'] = row['City/State'] + ', ' + row['Country']
        row['Country'] = 'United States'
    return row

# Aplicar la función a cada fila del DataFrame
df = df.apply(corregir_ciudad_estado, axis=1)

Separamos la columna Date en Year y Month

In [146]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df.insert(1, 'Year', df.pop('Year'))
df.insert(2, 'Month', df.pop('Month'))

Quitamos las columnas que no son relevantes para hacer análisis

In [147]:
df = df.drop(columns=['Flight', 'Registration', 'Serial', 'Passenger_Aboard', 'Crew_Aboard', 'Passenger_Fatalities', 'Ground_Fatalities', 'Location'])

Reacomodar las columnas para mejor visualización del dataset

In [148]:
# Extraer la última columna
last_column = df.pop(df.columns[-1])

position_to_insert = 4
df.insert(position_to_insert, last_column.name, last_column)

# Extraer la última columna
last_column = df.pop(df.columns[-1])

position_to_insert = 4
df.insert(position_to_insert, last_column.name, last_column)

Eliminamos las palabras 'Near'

In [149]:
# Eliminar 'Near ' de las columnas 'City/State' y 'Country'
df['City/State'] = df['City/State'].str.replace('Near ', '')
df['Country'] = df['Country'].str.replace('Near ', '')

Crear una columna que identifique si el operador del avión es militar o no

In [150]:
# Función para determinar si 'Operator' contiene palabras relacionadas con lo militar
def is_military(operator):
    if operator is None:
        return 'No'
    
    military_keywords = ['Military', 'Army', 'Navy', 'Force', 'Militar', 'Armada', 'Naval', 'Fuerza']
    for keyword in military_keywords:
        if keyword.lower() in str(operator).lower():
            return 'Yes'
    return 'No'

# Aplicar la función a cada fila y agregar los resultados a una nueva columna 'Military'
df['Military'] = df['Operator'].apply(is_military)

# Extraer la última columna
last_column = df.pop(df.columns[-1])

position_to_insert = 7
df.insert(position_to_insert, last_column.name, last_column)

In [151]:
# Función para verificar si la columna 'Summary' contiene palabras relacionadas con el clima
def is_weather_related(summary):
    if pd.isna(summary):  # Verificar si el campo está vacío
        return 'No'
    
    weather_keywords = ['rain', 'wind', 'snow', 'storm', 'sunny', 'cloudy', 'fog', 'hail', 'thunder', 'tornado',
                        'hurricane', 'blizzard', 'freezing', 'ice', 'sleet', 'heat', 'cold', 'flood', 'drought', 'weather']
    summary = summary.lower()  # Convertir a minúsculas para hacer la comparación insensible a mayúsculas
    for keyword in weather_keywords:
        if keyword in summary:
            return 'Yes'
    return 'No'

# Aplicar la función a cada fila de la columna 'Summary' y crear la columna 'Weather'
df['Weather_Accident'] = df['Summary'].apply(is_weather_related)

In [152]:
df_final = df

In [153]:
df_final.to_csv('.\Accidentes_limpio.csv', index=False)

In [154]:
df_final

,Date,Year,Month,Time,City/State,Country,Operator,Military,Route,Type,Aboard,Fatalities,Crew_Fatalities,Summary,Weather_Accident
0,1908-09-17,1908,9,17:00,"Fort Myer, Virginia",United States,Military - U.S. Army,Yes,Demonstration,Wright Flyer III,2,1,0,"During a demonstration flight, a U.S. Army flyer flown by Orville Wright nos...",No
1,1909-09-07,1909,9,NaN,Juvisy-sur-Orge,France,NaN,No,Air show,Wright Byplane,1,1,0,"Eugene Lefebvre was the first pilot to ever be killed in an air accident, af...",No
2,1912-07-12,1912,7,06:00,"Atlantic City, New Jersey",United States,Military - U.S. Navy,Yes,Test flight,Dirigible,5,5,5,"First U.S. dirigible Akron exploded just offshore at an altitude of 1,000 ft...",No
3,1913-08-06,1913,8,NaN,"Victoria, British Columbia",Canada,Private,No,NaN,Curtiss seaplane,1,1,1,The first fatal airplane accident in Canada occurred when American barnstorm...,Yes
4,1913-09-09,1913,9,18:00,Over the North Sea,None,Military - German Navy,Yes,NaN,Zeppelin L-1 (airship),20,14,0,The airship flew into a thunderstorm and encountered a severe downdraft cras...,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,2021-03-28,2021,3,18:00,"Butte, Alaska",United States,Soloy Helicopters,No,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,6,5,1,"The sightseeing helicopter crashed after missing the top of a 6,000 ft mount...",No
5004,2021-05-21,2021,5,18:00,Kaduna,Nigeria,Military - Nigerian Air Force,Yes,NaN,Beechcraft B300 King Air 350i,11,11,4,"While on final approach, in poor weather conditions, the aircraft crashed an...",Yes
5005,2021-06-10,2021,6,08:00,Pyin Oo Lwin,Myanmar,Military - Myanmar Air Force,Yes,Naypyidaw - Anisakan,Beechcraft 1900D,14,12,1,The plane was carrying military personnel and monks when it crashed about 30...,Yes
5006,2021-07-04,2021,7,NaN,"Patikul, Sulu",Philippines,Military - Philippine Air Force,Yes,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,96,50,0,"While attempting to land at Jolo Airport, the military transport overran the...",No
